# Imports

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations

from collections import defaultdict

import tensorflow as tf
from tensorflow.keras import layers, models, Input, callbacks

from matplotlib import pyplot as plt

import copy

from functions import  create_set, decreasing_signal, make_predictions, posHeatmapXY, default_to_regular, spatial_plot, generate_pdda_preds, lsq

# Dataset Import

In [ ]:
wd = '..' # path to data

In [ ]:
rooms = ['testbench_01', 'testbench_01_furniture_low', 'testbench_01_furniture_mid', 'testbench_01_furniture_high']
concrete_rooms = ['testbench_01_furniture_low_concrete', 'testbench_01_furniture_mid_concrete', 'testbench_01_furniture_high_concrete']
other_scenarios = ['testbench_01_scenario2', 'testbench_01_scenario3']
anchors = ['anchor1', 'anchor2', 'anchor3', 'anchor4']
channels = ['37','38','39']
polarities = ['V','H']

In [ ]:
data = defaultdict(lambda: defaultdict(lambda: defaultdict (lambda: defaultdict(list))))
anchor_data = defaultdict(lambda: defaultdict(lambda: defaultdict (lambda: defaultdict(list))))
for room in rooms + concrete_rooms + other_scenarios: 
    for channel in channels:  
        for polarity in polarities: 
             
            tag_filename = f'{wd}/data/{room}/tag_ml_export_CH{channel}_{polarity}.json'
            tag_df = pd.read_json(tag_filename, orient='records')

            anchor_filename = f'{wd}/data/{room}/anchor_ml_export_CH{channel}_{polarity}.json'
            anchor_df = pd.read_json(anchor_filename, orient='records')

            df = tag_df.merge(anchor_df)
            df.drop(df[(df['x_tag']==0).values | (df['y_tag']==0).values | (df['z_tag']==0).values].index, inplace=True)
            for anchor in anchors:
                data[room][anchor][channel][polarity] = df[df['anchor']==int(anchor[-1])]
                anchor_data[room][anchor][channel][polarity] = anchor_df

# Data Processing

In [ ]:
#Point sets creation
uniform_points = True
if uniform_points:

    points = data['testbench_01']['anchor1']['37']['H'].iloc[:, 1:7]

    # We use only the tag locations common in all rooms
    for room in rooms + concrete_rooms:
        for anchor in anchors:
            for channel in channels:
                for polarization in ['H','V']:
                    points = pd.merge(points, data[room][anchor][channel][polarization]['point'], on='point')
    
    xs = sorted(np.unique(points['x_tag']))[::6]
    ys = sorted(np.unique(points['y_tag']))[::3]
    train_points = points[points['x_tag'].isin(xs) & points['y_tag'].isin(ys)]
    
    xs = sorted(np.unique(points['x_tag']))[3::10]
    ys = sorted(np.unique(points['y_tag']))[3::10]
    val_points = points[points['x_tag'].isin(xs) & points['y_tag'].isin(ys)]

    test_points = points.drop(index=train_points.index).drop(index=val_points.index)

In [ ]:
print(f'Train Points No.: {len(train_points)}')
print(f'Val Points No.: {len(val_points)}')
print(f'Test Points No.: {len(test_points)}')

In [ ]:
plt.scatter(train_points.iloc[:,1:2].values,train_points.iloc[:,2:3].values)
plt.title('Training points')
plt.show()

We have 5 IQ measurements for each anchor-tag pair, one for each antenna of the anchor. We use the angle of the IQ-value of the first antenna element as reference for the rest of the antnnas. We also use the calculated RSSI value (signal strength) as feature for each anchor-tag pair.

The following are datasets that only use the polarization with the higher signal strength (RSSI) for each point.

In [ ]:
#train/val/test sets creation
x_train, y_train = create_set(data, train_points)

x_val, y_val = create_set(data, val_points)
                              
x_test, y_test = create_set(data, test_points)

In [ ]:
#data augmentation in training set
x_reduced_signal_amp = []

for i in range(30):
    x_reduced_signal_amp.append(decreasing_signal(x_train, rooms, scale_util=5))

x_train_reduced_amp = copy.deepcopy(x_train)
y_train_reduced_amp = copy.deepcopy(y_train)

for room in rooms:
    for anchor in anchors:
        for channel in channels:
            for i in range(30):
                x_train_reduced_amp[room][anchor][channel] = pd.concat([x_train_reduced_amp[room][anchor][channel], x_reduced_signal_amp[i][room][anchor][channel]])
                
            for i in range(30):
                y_train_reduced_amp[room][anchor][channel] = pd.concat([y_train_reduced_amp[room][anchor][channel], y_train[room][anchor][channel]])

# Triplet of Anchors AoA Estimation


## Model

In [ ]:
def aoa_mlp(anchors, lr, arch=[[24], [0.1]]):

    '''
    Input: 3 x (8 IQ Values, 1 x RSSI, 1 x IQ Reference Values) (for three anchors)
    Output: A latence space 9 x 1
    '''

    inp = {}
    x = []
    for anchor in anchors:
        inp[anchor] = Input((10,))
        x.append(inp[anchor])

    x = layers.concatenate(x)
    x = layers.BatchNormalization()(x)
    for i in range(len(arch[0])):
        x = layers.Dense(arch[0][i], activation='relu')(x)
        x = layers.Dropout(arch[1][i])(x)

    out = layers.Dense(9)(x)

    model = tf.keras.Model(
        inputs=inp,
        outputs=out,
    )

    return model

In [ ]:
def smol_mlp(lr):

    '''
    Input: The outputs of the aoa_mlps concatenated
    Output: A latence space 12x1 
    '''

    inp1 = Input((27,), name='AoAs')
    inp2 = Input((9), name='Powers ')

    x = layers.concatenate([inp1, inp2])

    out = layers.Dense(27)(x)
    out = layers.Dropout(0.15)(out)
    out = layers.Dense(12)(out)
    
    model = tf.keras.Model(
        inputs=[inp1, inp2],
        outputs=out,
    )

    return model

In [ ]:
def three_anchor_model(lr=0.01):
    '''
    Input: 3 x 4 x (8 IQ Values, 1 x RSSI, 1 x IQ Reference Values) (for the 4 anchors and the 3 channels)
    Output: The 8 predicted Azimuthian and Elevation AoAs of the four anchors.
    '''
    combs = list(combinations(anchors,3))
    
    x = defaultdict(dict)
    p = defaultdict(dict)
    for channel in channels:
        for anchor in anchors:
            x[anchor][channel] = Input((10,), name=f'{anchor}-{channel}')
            p[anchor][channel] = layers.Lambda(lambda x: x[:,1:2])(x[anchor][channel])

    y = defaultdict(list)
    w = []
    for i,comb in enumerate(combs):
        for channel in channels:
            y[i].append(aoa_mlp(comb, lr=lr)([x[anchor][channel] for anchor in comb]))

        y[i] = layers.concatenate(y[i])
        powers = layers.concatenate([p[anchor][channel] for anchor in comb for channel in channels])
        y[i] = smol_mlp(lr=lr)([y[i], powers])
        w.append(y[i])
        w.append(powers)
    
    w = layers.concatenate(w)
    
    w = layers.Dropout(0.1)(w)
    w = layers.Dense(32, activation='relu')(w)
    w = layers.Dropout(0.1)(w)
    w = layers.Dense(16, activation='relu')(w)

    out = layers.Dense(8)(w)

    model = tf.keras.Model(
        inputs=x,
        outputs=out,
    )

    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=lr), loss = 'mse', metrics = 'mae')

    return model

In [ ]:
three_anchor_model(0.001).summary()

In [ ]:
aoa_mlp([1,2,3],0.001).summary()

In [ ]:
smol_mlp(0.001).summary()

## Optimizing

In [ ]:
#Train parameters
fit_params = {'batch_size': 128, 'validation_batch_size':32, 'epochs': 50, 'verbose': 1,
              'callbacks': [callbacks.EarlyStopping(monitor='val_mae', mode='min', verbose=0, patience=75, restore_best_weights=True)]}

In [ ]:
#Model training
load = False

if load:
    triplets_models = {}
    for room in rooms:
        triplets_models[room] = models.load_model(f'/path_to_model/triplets_model/{room}') #add your path

else:

    triplets_models = defaultdict(lambda: three_anchor_model(0.002))

    for training_room in rooms:

        print(training_room)
        triplets_models[training_room] = three_anchor_model(0.002)

        ytrain = pd.concat([y_train_reduced_amp[training_room][anchor]['37'] for anchor in anchors], axis=1)
        yval = pd.concat([y_val[training_room][anchor]['37'] for anchor in anchors], axis=1)

        triplets_models[training_room].fit(x_train_reduced_amp[training_room], ytrain,
                                    validation_data=(x_val[training_room], yval), 
                                    **fit_params)

In [ ]:
# You can save the model
save =  False

if save:
    for room in rooms:
        triplets_models[room].save(f'/path_to_model/triplets_model/{room}') #add your path

## Predictions

In [ ]:
# Make predictions
preds, true_pos = make_predictions(x_test, y_test, triplets_models, training_rooms=rooms, testing_rooms=rooms+concrete_rooms+other_scenarios,
                                     test_points=test_points, anchor_data = anchor_data)

triplets_pos_preds = preds['pos_preds']
triplets_pos_maes = preds['pos_maes']

In [ ]:
# You can save your predictions
save_preds = False
if save_preds:
    results_path = f'/path_to_preds/triplets_preds.npy' #add your path
    np.save(results_path, default_to_regular(preds))

In [ ]:
# You can load your predictions
load_preds = False
if load_preds:
    preds = np.load(f'/path_to_preds/triplets_preds.npy', allow_pickle=True)[()] #add your path

## PDDA

In [ ]:
pdda_res = generate_pdda_preds(data, test_points, anchor_data)

# Results

In [ ]:
posHeatmapXY(triplets_pos_maes[:,:7], pdda_res['pos_maes'][:7])

In [ ]:
spatial_plot(triplets_pos_preds['testbench_01_furniture_low']['testbench_01_furniture_high'], true_pos, testing_room = 'testbench_01_furniture_high', mode = 'xy',vmax = 3, cmap = 'YlOrRd')